In [1]:
# Python code snippet for optimizing MLPRegressor using Genetic Algorithm

from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from deap import base, creator, tools, algorithms
import random
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler


In [2]:
# Assuming X_train, X_test, y_train, y_test are defined and available
file_path = 'ML_photocatalysis2.xlsx'

# Read the Excel file
data = pd.read_excel(file_path, header=0)

# Assuming 'df' is your pandas DataFrame.
df = data.sample(frac=1, random_state=38).reset_index(drop=True)

# Separate the features and the target variable
X = df[['Ni to TiO2 ratio', 'gC3N4 to TiO2 ratio', 'dosage', 'Time']]
y = df['SA removal rate']

# Split the data into training, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Scaling the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [8]:

# Define the parameter grid to search
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 100), (50, 100, 50), (100, 50, 100)],
    'solver': ['adam'],
    'alpha': [0.0001, 0.001],
    'learning_rate_init': [0.0001, 0.001, 0.01],
    'max_iter': [1000, 2000, 3000],
}

# Initialize the MLPRegressor model
regr = MLPRegressor(random_state=42, max_iter=10000)
# regr = MLPRegressor(random_state=42, max_iter=10000, activation='logistic')
activation='tanh'
# Initialize the Grid Search model
grid_search = GridSearchCV(regr, param_grid, cv=5, scoring='neg_mean_squared_error', verbose=2, n_jobs=-1)

# Fit the model
grid_search.fit(X_train_scaled, y_train)

# Best model
best_model_MLP = grid_search.best_estimator_

# Predictions
y_pred_test = best_model_MLP.predict(X_test_scaled)
y_pred_train = best_model_MLP.predict(X_train_scaled)

# Evaluate the model
n_test = X_test_scaled.shape[0]  # Number of observations
p_test = X_test_scaled.shape[1]  # Number of predictors
test_mse = mean_squared_error(y_test, y_pred_test)
test_r2 = r2_score(y_test, y_pred_test)
test_adjusted_r_squared = 1 - (1 - test_r2) * (n_test - 1) / (n_test - p_test - 1)

n_train = X_train_scaled.shape[0]  # Number of observations
p_train = X_train_scaled.shape[1]  # Number of predictors
train_mse = mean_squared_error(y_train, y_pred_train)
train_r2 = r2_score(y_train, y_pred_train)
train_adjusted_r_squared = 1 - (1 - train_r2) * (n_train - 1) / (n_train - p_train - 1)

print(f'Best model parameters: {grid_search.best_params_}')
print(f'MSE_test: {test_mse}, R2_test: {test_r2}, Adjust_R2_test: {test_adjusted_r_squared}')
print(f'MSE_train: {train_mse}, R2_train: {train_r2}, Adjust_R2_train: {train_adjusted_r_squared}')

# Note: BIC calculation is omitted as it requires a specific function `calculate_log_likelihood_from_mse` and `em.bic`
# which are not standard in sklearn and need to be defined.


Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best model parameters: {'alpha': 0.001, 'hidden_layer_sizes': (50, 50), 'learning_rate_init': 0.01, 'max_iter': 1000, 'solver': 'adam'}
MSE_test: 0.0012766163811221791, R2_test: 0.9776072845197046, Adjust_R2_test: 0.9741622513688899
MSE_train: 0.001733954896552702, R2_train: 0.9653365043539711, Adjust_R2_train: 0.9641514275797479


In [12]:

# Fitness function to evaluate the performance of MLPRegressor with given hyperparameters
def evaluate_model(individual):
    # Decode GA individual to MLP parameters
    hidden_layer_sizes = [(50,), (100,), (50, 50), (100, 100), (50, 100, 50), (100, 50, 100)][individual[0]]
    alpha = [0.0001, 0.001][individual[1]]
    learning_rate_init = [0.0001, 0.001, 0.01][individual[2]]
    max_iter = [1000, 2000, 3000][individual[3]]

    # Create and fit the MLPRegressor
    mlp = MLPRegressor(hidden_layer_sizes=hidden_layer_sizes,
                       alpha=alpha,
                       learning_rate_init=learning_rate_init,
                       max_iter=max_iter,
                       random_state=42)
    mlp.fit(X_train_scaled, y_train)

    # Evaluate the model
    predictions = mlp.predict(X_test_scaled)
    mse = mean_squared_error(y_test, predictions)
    
    return (mse,)

# Genetic Algorithm setup
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("attr_hidden_layer_sizes", random.randint, 0, 5)
toolbox.register("attr_alpha", random.randint, 0, 1)
toolbox.register("attr_learning_rate_init", random.randint, 0, 2)
toolbox.register("attr_max_iter", random.randint, 0, 2)

toolbox.register("individual", tools.initCycle, creator.Individual, 
                 (toolbox.attr_hidden_layer_sizes, toolbox.attr_alpha,
                  toolbox.attr_learning_rate_init, toolbox.attr_max_iter), n=1)

toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evaluate_model)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

# Genetic Algorithm execution
population = toolbox.population(n=100)
ngen = 10  # Number of generations
result = algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=ngen, verbose=True)

# Extracting the best individual
best_ind = tools.selBest(population, k=1)[0]
print("Best Individual: ", best_ind)
print("Best MSE: ", best_ind.fitness.values[0])

# Train and evaluate the MLPRegressor with the best found hyperparameters
# This section would involve retraining the MLPRegressor using the best parameters found by the GA
# and evaluating its performance on the test set. The code structure would be similar to the fitness function.



/Users/xiaotongsun/opt/anaconda3/lib/python3.8/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/Users/xiaotongsun/opt/anaconda3/lib/python3.8/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals
0  	100   
1  	56    
2  	71    
3  	53    
4  	57    
5  	64    
6  	63    
7  	66    
8  	68    
9  	68    
10 	47    
Best Individual:  [2, 1, 2, 0]
Best MSE:  0.0012766163811221791
